In [1]:
import os 
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY = os.getenv("OPEN_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5)

C:\Users\DEEPAK YADAV\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [10]:
from tabnanny import verbose
quiz_chain = LLMChain(llm=llm , prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
# connect both chains by using sequential chain
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [15]:
file_path = r"C:\Users\DEEPAK YADAV\Downloads\mcqgen\data.txt"

In [16]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [17]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [18]:
#serialize the pyhton dictionary into json-formatted-string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [19]:
NUMBER=5 
SUBJECT="machine learning"
TONE="difficult"


In [20]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\DEEPAK YADAV\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [21]:
print(cb)

Tokens Used: 2503
	Prompt Tokens: 1794
	Completion Tokens: 709
Successful Requests: 2
Total Cost (USD): $0.004109


In [22]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [23]:
quiz = (response.get("quiz"))

In [24]:
json.loads(quiz)

{'1': {'mcq': 'What term was coined by Arthur Samuel in 1959 to describe the field of computer gaming and artificial intelligence?',
  'options': {'a': 'Neural Networks',
   'b': 'Machine Learning',
   'c': 'Artificial Intelligence',
   'd': 'Cybertron'},
  'correct': 'b'},
 '2': {'mcq': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'options': {'a': 'Arthur Samuel',
   'b': 'Tom M. Mitchell',
   'c': 'Donald Hebb',
   'd': 'Walter Pitts'},
  'correct': 'c'},
 '3': {'mcq': "In the 1960s, what experimental 'learning machine' was developed by Raytheon Company for analyzing sonar signals and speech patterns?",
  'options': {'a': 'Cybertron',
   'b': 'Neural Networks',
   'c': 'Teaching Strategies',
   'd': 'Learning Machines'},
  'correct': 'a'},
 '4': {'mcq': "Who provided a formal definition of machine learning algorithms stating 'A computer program learns from experience with respect to tasks and performance measure'?",
 

In [25]:
quiz=json.loads(quiz)

In [26]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [27]:
quiz_table_data

[{'MCQ': 'What term was coined by Arthur Samuel in 1959 to describe the field of computer gaming and artificial intelligence?',
  'Choices': 'a: Neural Networks | b: Machine Learning | c: Artificial Intelligence | d: Cybertron',
  'Correct': 'b'},
 {'MCQ': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'Choices': 'a: Arthur Samuel | b: Tom M. Mitchell | c: Donald Hebb | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': "In the 1960s, what experimental 'learning machine' was developed by Raytheon Company for analyzing sonar signals and speech patterns?",
  'Choices': 'a: Cybertron | b: Neural Networks | c: Teaching Strategies | d: Learning Machines',
  'Correct': 'a'},
 {'MCQ': "Who provided a formal definition of machine learning algorithms stating 'A computer program learns from experience with respect to tasks and performance measure'?",
  'Choices': 'a: Raytheon Company | b: Tom M. Mitchell | c: Donald Hebb | d: Walter Pit

In [28]:
quiz=pd.DataFrame(quiz_table_data)

In [29]:
quiz.to_csv("machinelearning.csv",index=False)

In [30]:
from datetime import datetime
datetime.now().strftime('On date: %m/%d/%Y , at time: %H:%M:%S')

'On date: 05/21/2024 , at time: 10:46:44'